# TAREFA DATASET GRUPO

Para esta fase, decidimos escolher um dataset referente a avaliações de vinhos. 
Pode ser encontrado neste link: https://www.kaggle.com/datasets/zynicide/wine-reviews <br>
O objetivo desta fase é, através deste dataset, inferir acerca do valor de points que será dado a um dado vinho.

## TODO:
* Resolver modelos que não estão a executar (e estão comentados) - demoram muito:
    * SVR
    * RN
    * Voting
    * Bagging
    * Blending
* Colocar dataset mais "limpo"

### **1.** Importar as bibliotecas essenciais do Python para a elaboração desta tarefa

In [ ]:
import sklearn as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('always')

### **2. Carregar o dataset para um dataframe da biblioteca Panda**

In [ ]:
df = pd.read_csv('docs/winemag-data-130k-v2.csv', encoding="utf-8", skipinitialspace=True)

### **3. Obtenção de informação acerca do dataset**

* **tipos de dados das features**

In [ ]:
df.info()

A feature objetivo, points, está preenchida em todas as linhas.<br>
Muitas das outras features têm de ser preenchidas mais tarde ou retiradas completamente por estarem incompletas (como taster_twiter_handle)

* **conteúdo do dataset**

In [ ]:
df

Reparamos que a feature Unnamed: 0 apenas serve como id, sendo portanto desnecessária. <br>
Region 1 e 2 sao apenas especificações de province, dependendo do seu número de valores únicos, talvez sejam um overload de informação que é melhor excluir. <br>
Da mesma forma, taster_twitter_handle apenas complementa taster_name.

* **estatística**

In [ ]:
df.describe()

Podemos ver que embora points seja da perspetiva de fora um atributo de classificação que iria de 0-100, neste dataset apenas temos valores entre 80 e 100.<br>
O valor máximo de preço é muito superior à sua média (possível necessidade de tratar de outliers).


* **Distribuição da feature target**

In [ ]:
df.points.hist()

Distribuição do target é aproximadamente normal, o que será benéfico para a criação de modelos, como por exemplo de regressão linear.

* **Distribuição da feature price**

In [ ]:
df.price.hist(bins=[0,20,50,100,200,300,1000,2000,2500,3000])

Através do grafo dos preços (possível segunda feature alvo), podemos ver que a grande maioria destes se distribui por volta de valores menores que 100.

* **Análise dos valores únicos das features**

In [ ]:
for c in df:
    print(f"{c}: {df[c].unique()}")
    print(f"Quantidade: {df[c].nunique()}")
    print("---------------------------------------")

| Feature               | Justificação                                                                                                                                          |
|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| Unnamed               | Como mencionado anteriormente, esta feature serve como id, devendo ser retirada.                                                                      |
| description           | O número de valores únicos é muito elevado, além de serem strings sem nenhum significado para os modelos                                              |
| designation           | Número de valores únicos muito elevados (para strings).                                                                                               |
| province              | Embora uma especificação de country, como não tem um número excessivo de valores únicos, pode ser benéfico.                                           |
| region_1              | Especificação de especificação (province), com muitos valores únicos. Provavelmente melhor retirar.                                                   |
| region_2              | Nova especificação de province.                                                                                                                       |
| taster_twitter_handle | Especificação/AlterEgo do taster_name, informação redundante.                                                                                         |
| title                 | Como mencionado anteriormente, número de únicos demasiado elevado, mas contém informação útil dentro deste, devendo ser retirada nos próximos passos. |
| variety               | Número elevado de strings únicas, pode ou não ser relevante, devendo ser experimentado o modelo em ambos os casos.                                    |
| winery                | Número muito elevado de strings únicas, embora potencialmente relevantes numa perspetiva de mundo real.                                               |

### **4. Preparação dos dados**

In [ ]:
df.head()

In [ ]:
df.columns

* **Remoção de features**

In [ ]:
remove_features_list = ["Unnamed: 0",'designation', "description",'region_1', "region_2", "taster_twitter_handle","winery"]
for ft in remove_features_list:
    df = df.drop(ft, axis=1)

* **Preenchimento valores em falta**

In [ ]:
df.price.fillna(df.price.mean(),inplace =True)
df.country.fillna(str(df.country.mode()),inplace =True)
df.province.fillna(str(df.province.mode()),inplace =True)
df.taster_name.fillna('unknown',inplace =True)
df.variety.fillna(str(df.variety.mode()),inplace =True)

* **Tratamento de title**

Obsevando previamente os valores da feature title, observamos que todos parecem incluir o ano de produção de vinho e, embora a atualmente esta feature apresente demasiados valores únicos para ser útil, se conseguirmos retirar apenas o ano desta, possívelmente poderá ser usado no modelo.

In [ ]:
df[df.title.str.contains(r'.*\d{4}.*')].info()

Confirmamos que a grande maioria das linhas apresenta o ano. (apenas 4609 de 130k não apresentam)

In [ ]:
df.title = df.title.str.replace(r'(.|\n)*(\d{4})(.|\n)*',r'\2',regex=True)
df.title = df.title[df.title.str.contains(r'\d{4}')]


In [ ]:
df.info()

Mudar nome da feature para year

In [ ]:
rename_map = {'title':'year'}
df.rename(columns=rename_map,inplace=True)

In [ ]:
df

Preencher valores nulos com a moda

In [ ]:
df.year.fillna(df.year.mode().astype(int).values[0],inplace=True)

In [ ]:
df.year = df.year.astype(int)

* **Remoção de duplicados**

In [ ]:
# Remoção de registos duplicados (caso hajam)
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

* **Labeling das features**

Número de valores únicos e tipo de cada feature

In [ ]:
columns = df.columns.values
for c in columns:
    print(f"{c} : {df[c].nunique()}  \n   type : {df[c].dtype}")

In [ ]:
df.info()

Labeling de features tipo object

In [ ]:
for c in df.columns.values:
    if(df[c].dtype=='object'):
        print(c)
        labels = df[c].astype('category').cat.categories.tolist()
        replace_map_comp = {c : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
        df.replace(replace_map_comp,inplace=True)

df.head()


### **5. Aplicação de modelos de Machine Learning**

#### **5.1. Decision Tree regressor**

* **Imports necessários**

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

* **Retirar target do data-set de treino e criar uma cópia do data-set, para verificar predictions**

In [ ]:
x = df.drop(['points'],axis=1)
y = df['points'].to_frame()

In [ ]:
x

In [ ]:
y

* **Separar o data-set em conjuntos de treino e teste** <br>
Tamanho de teste - 25% <br>
Seed = 2022

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

* **Criar modelo de árvores de decisão**

In [ ]:
clf = DecisionTreeRegressor(random_state=2022)

* **Treinar modelo**

In [ ]:
clf.fit(x_train,y_train)

Gerar previsões

In [ ]:
predictions = clf.predict(x_test)
pd.DataFrame(predictions)

MAE

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
y_test

#### **5.2. Linear Regression**

* **Importar as funções necessárias para este modelo**

In [ ]:
from sklearn.linear_model import LinearRegression

* **Separar o data-set em conjuntos de treino e teste** <br>
Tamanho de teste - 30% <br>
Seed = 2022

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

* **Criar modelo de regressão linear**

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(x_train, y_train)

Gerar previsões

In [ ]:
predictions = lm.predict(x_test)
df_pred = pd.DataFrame(data=predictions)
df_pred

In [ ]:
y_test

* **Avaliação do modelo**

In [ ]:
from sklearn import metrics
from math import sqrt

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

#### **5.3. Neural Network**

* **Importar funções necessárias para o modelo**

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
RANDOM_SEED = 2021

* **Escalar dados para o bom funcionamento da rede**

In [ ]:
scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(x)
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(y)
x_scaled = pd.DataFrame(scaler_X.transform(x[x.columns]), columns=x.columns)
y_scaled = pd.DataFrame(scaler_y.transform(y[y.columns]), columns=y.columns)

In [ ]:
x_scaled.head()

In [ ]:
y_scaled.head()

* **Separar dados de treino e de teste**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled,test_size=0.2,random_state=2022)

* **Criar modelo para as redes**

In [ ]:
def build_model(activation='relu', learning_rate=0.01):
    #Create a sequential model (with three Layers - Last one is the output)
    model = Sequential()
    model.add(Dense(16, input_dim=6, activation=activation) )
    model.add(Dense(8, activation=activation) )
    model.add(Dense(1, activation='relu'))

    #Compile the model
    #Define the Loss function, the otimizer and metrics to be used
    model .compile(
    loss = 'mae',
    optimizer = tf.optimizers.Adam(learning_rate),
    metrics = ['mae', 'mse'])
    return model

In [ ]:
model = build_model()

Tunning do modelo

In [ ]:
TUNING_DICT = {
'activation' : ['relu', 'sigmoid'],
'learning_rate' : [0.01, 0.001]
}

* **Correr modelo**

In [ ]:
#kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
#
#model = KerasRegressor(build_fn=build_model, epochs=20, batch_size=32)
#grid_search = GridSearchCV(estimator = model,
#                            param_grid = TUNING_DICT,
#                            cv = kf,
#                            scoring = 'neg_mean_absolute_error',
#                            refit = 'True',
#                            verbose = 1)
#
#grid_search.fit(x_train, y_train, validation_split=0.2)

* **Resultados**

In [ ]:
#summarize results
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Our best model (remember we set refit=True?)
best_mlp_model = grid_search.best_estimator_

In [ ]:
from livelossplot import PlotLossesKerasTF

**Fitting do modelo**

In [ ]:
best_mlp_model.fit(x_train, y_train, epochs=20,
validation_data=(x_test, y_test),
callbacks=[PlotLossesKerasTF()], verbose=1)

O fit não é o melhor, sugerindo que deveriamos aumentar a quantidade de dados de treino

* **Unscaled results**

In [ ]:
#Obtain predictions

predictions = best_mlp_model.predict(x_test)

predictions = predictions.reshape(predictions.shape[0], 1)
predictions[:5]

In [ ]:
#And now Let's unscale the model's predictions to see real prices!
predictions_unscaled = scaler_y.inverse_transform(predictions)
pd.DataFrame(predictions_unscaled)

In [ ]:
y_test_unscaled = pd.DataFrame(scaler_y.inverse_transform(y_test))
y_test_unscaled

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test_unscaled, predictions_unscaled))
print('MSE:', metrics.mean_squared_error(y_test_unscaled, predictions_unscaled))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_unscaled, predictions_unscaled)))

#### **5.4. Support Vector Machine**

Separar o data-set em conjuntos de treino e teste <br>
Tamanho de teste - 25% <br>
Seed = 2022

In [ ]:
from sklearn.model_selection import train_test_split
x = df.drop(['points'],axis=1)
y = df['points'].to_frame()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

Importar funções necessárias para este modelo

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

Criação de uma instância SVR()

In [ ]:
#model = SVR(kernel = 'rbf')
#model.fit(x_train,y_train.values.ravel())

Geração de previsões

In [ ]:
#y_pred = model.predict(x_test)
#y_pred

In [ ]:
y_test

**Avaliação do modelo**

Importar funções necessárias

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix

Matriz de confusão

In [ ]:
plot_confusion_matrix(model, x_test, y_test) 

Classification report

In [ ]:
print(classification_report(y_test, y_pred))

### 5.5 Ensemble learning

### 5.5.1 Voting

In [ ]:
#from sklearn.ensemble import VotingRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVR
#from sklearn.model_selection import train_test_split
#
#x = df.drop(['points'],axis=1)
#y = df['points'].to_frame()
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)
#
#
#best_svr = SVR()
#best_dt = DecisionTreeRegressor(random_state=2022)
#best_lr = LinearRegression()
#
#estimators = [('svr', best_svr), ('dt', best_dt), ('lr', best_lr)]
#vr = VotingRegressor(estimators, weights = [1, 1, 1])
#
#vr = vr.fit(x_train, y_train.values.ravel())


### 5.5.2 Bagging

In [ ]:
#from sklearn.ensemble import BaggingRegressor
#from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
#from sklearn.model_selection import train_test_split
#
#x = df.drop(['points'],axis=1)
#y = df['points'].to_frame()
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)
#
#n_estimators = [10,40,68, 80, 100, 160]
#
#cv = StratifiedShuffleSplit(n_splits=10, test_size=21, random_state=2022)
#parameters={'n_estimators' :n_estimators}
#
#gs = GridSearchCV(BaggingRegressor(DecisionTreeRegressor(random_state=2022), bootstrap = True),
#                  param_grid=parameters, cv=cv)
#gs = gs.fit(x_train, y_train.values.ravel())

In [ ]:
#predictions = gs.predict(x_test)
#mean_absolute_error(y_test,predictions)

### 5.5.3 Stacking

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

x = df.drop(['points'],axis=1)
y = df['points'].to_frame()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

boost = xgb.XGBRegressor(random_state=2022, verbosity=0, use_label_encoder=False)
forest = RandomForestRegressor(n_estimators=100,random_state=2022)
lgregression = LinearRegression(random_state=2022)

estimators = [
     ('rf', forest),
     ('xgb', boost)
]
sclf = StackingRegressor(estimators=estimators,
                          final_estimator=lgregression,
                          cv=10)

sclf.fit(x_train, y_train.values.ravel())

### 5.5.4 Blending

In [ ]:
#from sklearn.model_selection import train_test_split
#from sklearn.svm import SVR
#x = df.drop(['points'],axis=1)
#y = df['points'].to_frame()
#
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2022)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)
#
#model_1= SVR(gamma="scale")
#model_1.fit(X_train, y_train.values.ravel())
#
#val_pred_1= model_1.predict(X_val)
#test_pred_1= model_1.predict(X_test)
#
#model_2= RandomForestRegression(random_state=2022)
#model_2.fit(X_train, y_train.values.ravel())
#
#val_pred_2= model_2.predict(X_val)
#test_pred_2= model_2.predict(X_test)
#
#df_val= pd.concat([X_val, val_pred_1, val_pred_2], axis=1)
#df_test= pd.concat([X_test, test_pred_1, test_pred_2], axis=1)
#
#model = LinearRegression(random_state=2022)
#
#model.fit(df_val, y_val.values.ravel())

### 5.5.5 Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

x = df.drop(['points'],axis=1)
y = df['points'].to_frame()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2022)

clf = AdaBoostRegressor(random_state=2022)
clf.fit(x_train, y_train.values.ravel())

In [ ]:
predictions = rfc.predict(x_test)
mean_absolute_error(y_test,predictions)

### 5.5.6 XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

x = df.drop(['points'],axis=1)
y = df['points'].to_frame()
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=2022)

xg_reg = xgb.XGBRegressor(tree_method="hist",eval_metric=mean_absolute_error,random_state=2022, n_estimators=100)
xg_reg.fit(x_train, y_train.values.ravel())

In [ ]:
predictions = rfc.predict(x_test)
mean_absolute_error(y_test,predictions)

### 5.5.7 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
x = df.drop(['points'],axis=1)
y = df['points'].to_frame()
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=2022)

rfc = RandomForestRegressor(n_estimators=100, random_state=2022)
rfc.fit(x_train, y_train.values.ravel())

In [ ]:
predictions = rfc.predict(x_test)
mean_absolute_error(y_test,predictions)

*Usar GPU em vez de CPU para as redes neuronais*

In [ ]:
!pip install --upgrade tensorflow-gpu==2.4.1

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()
